<a href="https://colab.research.google.com/github/sachinthadilshann/TensorFlow_for_Deep_Learning_Bootcamp_Udemy/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
 !wget https://raw.githubusercontent.com/sachinthadilshann/TensorFlow_for_Deep_Learning_Bootcamp_Udemy/15a269df7d105222999c850ce9fa956af7610972/helper_functions.py

 from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-07-22 07:23:26--  https://raw.githubusercontent.com/sachinthadilshann/TensorFlow_for_Deep_Learning_Bootcamp_Udemy/15a269df7d105222999c850ce9fa956af7610972/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5351 (5.2K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]   5.23K  --.-KB/s    in 0s      

2025-07-22 07:23:26 (52.0 MB/s) - ‘helper_functions.py.1’ saved [5351/5351]



In [28]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

unzip_data("nlp_getting_started.zip")

--2025-07-22 07:23:26--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.179.207, 64.233.180.207, 192.178.155.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.179.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2025-07-22 07:23:26 (118 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [29]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [30]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [31]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [32]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [33]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [34]:
import random

random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n ")
  print("---\n")

Target: 0 (not real disaster)
Text:
http://t.co/eHKLp12yiP Paci?c Media Centre | articles: AUSTRALIA: RSF protests over new security gag over reporting on...
 
---

Target: 0 (not real disaster)
Text:
@RickyBonesSXM fuck u2 specially there new Shit. First they hijack itunes now all sirius chan. fuck off already haha just sayin
 
---

Target: 1 (real disaster)
Text:
DHS Refuses to Call Chattanooga Û÷Islamic TerrorismÛª out of respect for MUSLIMS ... http://t.co/u8RGB51d22 via @po_st http://t.co/2tnu95VGFE
 
---

Target: 0 (not real disaster)
Text:
That was crushed holy shit
 
---

Target: 1 (real disaster)
Text:
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/NX2d83A4Du
 
---



In [35]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                    train_df_shuffled["target"].to_numpy(),
                                                    test_size=0.1)

In [36]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [37]:
train_sentences[:3], train_labels[:3]

(array(['Amazon Prime Day: 12 quick takeaways from Amazon\x89Ûªs magnificent train wreck - http://t.co/DBDwtOcGXF',
        'Still Uninvestigated After 50 Years: Did the U.S. Help Incite the 1965 Indonesia Massacre? http://t.co/EZbTG81trz',
        'That moment when you get on a scary roller coaster and the guy behind you is just screaming bloody murder ?????? #silverwood #aftershock'],
       dtype=object),
 array([0, 1, 0]))

## Converting text into numbers

### Text vectorization (tokenization)


In [38]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [39]:
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?


In [40]:
 #Find average number of tokens(words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [41]:
max_vocab_length = 10000 # max #words to have in our vocabulary
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [42]:
text_vectorizer.adapt(train_sentences)

In [44]:
sample_sen =  "Hello world"

vectorized_sentence = text_vectorizer([sample_sen])
vectorized_sentence

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1403,  104,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [51]:
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Just demolished a Snowball ??

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  30,  630,    3, 4580,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>